## Elasticsearch: The Definitive Guide - Python

Following the examples in the book, here are Python snippets that achieve the same effect.

Documentation for the Python libs:

Low-level API:

https://elasticsearch-py.readthedocs.io/en/master/index.html

Expressive DSL API (more "Pythonic")

http://elasticsearch-dsl.readthedocs.io/en/latest/index.html

Github repo for DSL API:

https://github.com/elastic/elasticsearch-dsl-py


In [20]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q

es = Elasticsearch(
    'localhost',
    # sniff before doing anything
    sniff_on_start=True,
    # refresh nodes after a node fails to respond
    sniff_on_connection_fail=True,
    # and also every 60 seconds
    sniffer_timeout=60
)

### Empty Search
From: https://www.elastic.co/guide/en/elasticsearch/guide/master/empty-search.html

>GET _search

In [23]:
res = es.search('_all') # same as es.search()

In [25]:
#from pprint import pprint
#pprint(res)

In [33]:
s = Search(using=es)
response = s.execute()
response

<Response: [<Hit(.kibana/config/5.2.2): {'discover:aggs:terms:size': 20, 'defaultIndex': 'test', 'bu...}>, <Hit(.kibana/index-pattern/test): {'fields': '[{"name":"tags","type":"string","count":0,"scrip...}>, <Hit(.kibana/index-pattern/megacorp): {'fields': '[{"name":"last_name.keyword","type":"string","co...}>, <Hit(.kibana/index-pattern/website): {'fields': '[{"name":"date","type":"date","count":0,"scripte...}>, <Hit(us/tweet/14): {'date': '2014-09-24', 'tweet': 'How many more cheesy tweets...}>, <Hit(gb/tweet/5): {'date': '2014-09-15', 'tweet': 'However did I manage before...}>, <Hit(gb/tweet/9): {'date': '2014-09-19', 'tweet': 'Geo-location aggregations a...}>, <Hit(us/tweet/8): {'date': '2014-09-18', 'name': 'John Smith', 'user_id': 1}>, <Hit(us/tweet/10): {'date': '2014-09-20', 'tweet': 'Elasticsearch surely is one...}>, <Hit(us/tweet/12): {'date': '2014-09-22', 'tweet': 'Elasticsearch and I have le...}>]>

With timeout:

>GET /_search?timeout=10ms

In [36]:
res = es.search('_all', timeout='10ms') # same as es.search(timeout='10ms')

In [71]:
# To see the results, we can iterate:
# Elasticsearch pages the results (to 10 hits)
for hit in s:
    print(hit)

<Hit(us/tweet/14): {'date': '2014-09-24', 'tweet': 'How many more cheesy tweets...}>
<Hit(gb/tweet/5): {'date': '2014-09-15', 'tweet': 'However did I manage before...}>
<Hit(gb/tweet/9): {'date': '2014-09-19', 'tweet': 'Geo-location aggregations a...}>
<Hit(us/tweet/8): {'date': '2014-09-18', 'name': 'John Smith', 'user_id': 1}>
<Hit(us/tweet/10): {'date': '2014-09-20', 'tweet': 'Elasticsearch surely is one...}>
<Hit(us/tweet/12): {'date': '2014-09-22', 'tweet': 'Elasticsearch and I have le...}>
<Hit(gb/user/2): {'email': 'mary@jones.com', 'name': 'Mary Jones', 'username'...}>
<Hit(us/tweet/4): {'date': '2014-09-14', 'tweet': '@mary it is not just text, ...}>
<Hit(us/tweet/6): {'date': '2014-09-16', 'tweet': 'The Elasticsearch API is re...}>
<Hit(gb/tweet/7): {'date': '2014-09-17', 'tweet': 'The Query DSL is really pow...}>


### Multi-index, Multitype

First using the low-level API

In [63]:
#/_search
#Search all types in all indices
res = es.search('_all')

#/gb/_search
#Search all types in the gb index
res = es.search(index='gb')

#/gb,us/_search
#Search all types in the gb and us indices
res = es.search(index=['gb','us'])

#/g*,u*/_search
#Search all types in any indices beginning with g or beginning with u
res = es.search(index=['g*','u*'])

#/gb/user/_search
#Search type user in the gb index
res = es.search(index='gb', doc_type='user')

#/gb,us/user,tweet/_search
#Search types user and tweet in the gb and us indices
res = es.search(index=['g*','u*'], doc_type=['user', 'tweet'])
print(res['hits']['total'])

#/_all/user,tweet/_search
#Search types user and tweet in all indices
res = es.search(doc_type=['user', 'tweet'])
print(res['hits']['total'])

14
14


Next using the DSL, although similar for such basic searches

In [75]:
#/_search
#Search all types in all indices
s = Search(using=es)
response = s.execute()

#/gb/_search
#Search all types in the gb index
s = Search(using=es, index='gb')
response = s.execute()

#/gb,us/_search
#Search all types in the gb and us indices
s = Search(using=es, index=['gb','us'])
response = s.execute()

#/g*,u*/_search
#Search all types in any indices beginning with g or beginning with u
s = Search(using=es, index=['g*','u*'])
response = s.execute()

#/gb/user/_search
#Search type user in the gb index
s = Search(using=es, index=['g*','u*'], doc_type='user')
response = s.execute()


#/gb,us/user,tweet/_search
#Search types user and tweet in the gb and us indices
s = Search(using=es, index=['g*','u*'], doc_type=['user','tweet'])
response = s.execute()

#/_all/user,tweet/_search
#Search types user and tweet in all indices
s = Search(using=es, doc_type=['user','tweet'])
response = s.execute()
print(response['hits']['total'])
print(len(res['hits']['hits']))

14
10


### Pagination

The last search produced a hits total of 14, but there are only 10 documents in the array.

This is due to pagination, so we need to use pointers:

>GET /_search?size=5

>GET /_search?size=5&from=5

>GET /_search?size=5&from=10


In [78]:
# For search API:
res = es.search(doc_type=['user', 'tweet'], from_=5, size=5)

In [80]:
print(res['hits']['total'])
print(len(res['hits']['hits']))

14
5


### Search Lite

These initial searches all use the Lucene Query String Syntax.

>GET /_all/tweet/_search?q=tweet:elasticsearch

For the low-level API, we use the q parameter:

In [88]:
res = es.search(doc_type=['tweet'], q='tweet:elasticsearch')
print('Total hits:{}\n'.format(res['hits']['total']))
pprint(res['hits']['hits'][0])

Total hits:7

{'_id': '13',
 '_index': 'gb',
 '_score': 0.7081689,
 '_source': {'date': '2014-09-23',
             'name': 'Mary Jones',
             'tweet': 'So yes, I am an Elasticsearch fanboy',
             'user_id': 2},
 '_type': 'tweet'}


For the DSL, the intended purpose is to avoid the query string syntax and use the query string language instead. For completeness, here is an equivalent script:

In [96]:
s = Search(using=es, doc_type=['tweet']) \
    .query('match', tweet='elasticsearch')
response = s.execute()
print('Total hits:{}\n'.format(response['hits']['total']))
pprint(response['hits']['hits'][0])

Total hits:7

{'_source': {'date': '2014-09-23', 'tweet': 'So yes, I am an...}


However, notice that the pprint has not given us the same JSON response as the above query string syntax result via the low-level API. This is because the Search() object returns an array of Hit objects. These are constructed so as to expose the individual fields as object attributes (__getattr__)

In [120]:
for hit in response:
    print(hit.tweet)

So yes, I am an Elasticsearch fanboy
However did I manage before Elasticsearch?
The Elasticsearch API is really easy to use
Elasticsearch surely is one of the hottest new NoSQL products
Elasticsearch means full text search has never been so easy
Elasticsearch is built for the cloud, easy to scale
Elasticsearch and I have left the honeymoon stage, and I still love her.


### The _all field

> GET /_search?q=mary

In [121]:
res = es.search(q='mary')
print('Total hits:{}\n'.format(res['hits']['total']))
pprint(res['hits']['hits'][0])

Total hits:8

{'_id': '4',
 '_index': 'us',
 '_score': 0.6650044,
 '_source': {'date': '2014-09-14',
             'name': 'John Smith',
             'tweet': '@mary it is not just text, it does everything',
             'user_id': 1},
 '_type': 'tweet'}


For the DSL, we need to call the _all field explicitly

In [133]:
s = Search(using=es) \
    .query('match', _all='mary')
response = s.execute()
print('Total hits:{}\n'.format(response['hits']['total']))
print(response[0].tweet)

Total hits:8

@mary it is not just text, it does everything


> +name:(mary john) +date:>2014-09-10 +(aggregations geo)

In [134]:
res = es.search(q='+name:(mary john) +date:>2014-09-10 +(aggregations geo)')
print('Total hits:{}\n'.format(res['hits']['total']))
pprint(res['hits']['hits'][0])

Total hits:1

{'_id': '9',
 '_index': 'gb',
 '_score': 2.3835227,
 '_source': {'date': '2014-09-19',
             'name': 'Mary Jones',
             'tweet': 'Geo-location aggregations are really cool',
             'user_id': 2},
 '_type': 'tweet'}


In [159]:
from elasticsearch_dsl import A
a = A('terms', field='geo')
s = Search(using=es) \
    .query('multi_match', query='mary john', fields='_all') \
    .filter('range', date={'gt': '2014-09-10'}) \
    .aggs.bucket(name='my_agg', agg_type='terms', field='geo')
response = s.execute()
print('Total hits:{}\n'.format(response['hits']['total']))
print('tweet:{}, date:{}'.format(response[0].tweet, response[0].date))

AttributeError: 'Terms' object has no attribute 'execute'

In [ ]:
#"range": { "publish_date": { "gte": "2015-01-01" }